In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
res_2_df = pd.read_csv('results_1.2.csv')

In [18]:
os.makedirs("plots", exist_ok=True)

# Normalize dataframe
df = res_2_df.copy()
df['user'] = df.get('user', 'unknown')
df['method'] = df['method'].astype(str).str.strip().str.lower()
df['threads'] = pd.to_numeric(df.get('threads'), errors='coerce')
df.loc[df['method'] == 'sequential', 'threads'] = 0
df['threads'] = df['threads'].fillna(1).astype(int)
df['iterations'] = pd.to_numeric(df.get('iterations'), errors='coerce').fillna(0).astype(int)

time_cols = ["time_compute", "time_total", "time_thread_create"]
eps = 1e-12
for col in time_cols:
    df[col] = pd.to_numeric(df.get(col), errors='coerce').fillna(0) + eps

# Aggregate means/std
agg = (
    df.groupby(["user", "method", "iterations", "threads"])[time_cols]
      .agg(["mean", "std"])
      .reset_index()
)
agg.columns = ["_".join(c).strip("_") for c in agg.columns]

# Plot per user + combined totals
users = sorted(df["user"].unique())
if len(users) > 1:
    users.append("total")
    
colors = {"mutex":"#1f77b4","rwlock":"#ff7f0e","atomic":"#2ca02c","sequential":"#7f7f7f"}
hatches = ["xx", "\\\\"]

def iter_label(n):
    return f"{n//1_000_000}M" if n>=1_000_000 else f"{n//1000}K" if n>=1000 else str(n)

for user in users:
    user_df = df if user == "total" else df[df["user"] == user]
    user_agg = agg if user == "total" else agg[agg["user"] == user]

    if user_df.empty:
        continue

    methods = sorted(user_df["method"].unique(), key=lambda m: 0 if m=="sequential" else 1)
    iterations = sorted(user_df["iterations"].unique())
    threads = sorted(user_df["threads"].unique())

    # Pre-calculate available bars per thread
    bars_per_thread = {}
    max_bars = 0
    
    for thread in threads:
        thread_data = user_agg[user_agg["threads"] == thread]
        available_combinations = []
        
        for method in methods:
            for it in iterations:
                if not thread_data[(thread_data["method"] == method) & (thread_data["iterations"] == it)].empty:
                    available_combinations.append((method, it))
        
        bars_per_thread[thread] = available_combinations
        max_bars = max(max_bars, len(available_combinations))

    # Dynamic spacing calculation based on maximum bars
    x = np.arange(len(threads))
    width = 0.8 / max_bars if max_bars > 0 else 0.1
    spacing_scale = 0.9  # Slight adjustment for visual appeal

    fig, axes = plt.subplots(1, 2, figsize=(max(12, len(threads) * 1.5), 6))
    fig.suptitle(f"Shared Variable Update — {user}", fontsize=14, fontweight="bold")
    legend_handles, legend_labels = [], []
    
    alpha_levels = np.linspace(0.45, 0.95, len(iterations))
    alpha_map = {it: alpha_levels[i] for i, it in enumerate(iterations)}

    for thread_idx, thread in enumerate(threads):
        available_bars = bars_per_thread[thread]
        thread_offset_base = (-len(available_bars)/2 * width + width/2) * spacing_scale
        
        for bar_idx, (method, it) in enumerate(available_bars):
            sel = user_agg[(user_agg["method"] == method) & 
                          (user_agg["iterations"] == it) & 
                          (user_agg["threads"] == thread)]
            
            if sel.empty:
                continue

            y_compute = sel["time_compute_mean"].iloc[0]
            y_total = sel["time_total_mean"].iloc[0]
            offset = thread_offset_base + bar_idx * width
            label = f"{method} ({iter_label(it)})"
            color = colors.get(method, "#444")

            bars = axes[0].bar(
                thread_idx + offset, y_compute, width,
                color=color, alpha=alpha_map[it],
                hatch=hatches[bar_idx % len(hatches)],
                label=label
            )
            axes[1].bar(
                thread_idx + offset, y_total, width,
                color=color, alpha=alpha_map[it],
                hatch=hatches[bar_idx % len(hatches)]
            )

            if label not in legend_labels:
                legend_handles.append(bars)
                legend_labels.append(label)

    for ax, title, ylabel in zip(axes, ["Compute Time", "Total Time"], ["Compute Time (s)", "Total Time (s)"]):
        ax.set_title(title)
        ax.set_xlabel("Number of Threads")
        ax.set_ylabel(ylabel)
        ax.set_xticks(x)
        ax.set_xticklabels(threads)
        ax.grid(True, alpha=0.3)
        ax.set_yscale("log")
        ax.set_xlim(-0.5, len(threads))

    # Remove duplicate legend entries
    unique_handles = []
    unique_labels = []
    for handle, label in zip(legend_handles, legend_labels):
        if label not in unique_labels:
            unique_handles.append(handle)
            unique_labels.append(label)

    axes[1].legend(unique_handles, unique_labels, bbox_to_anchor=(1.05,1), loc="upper left")
    outpath = f"plots/plot_1_2_{user}.png"
    fig.savefig(outpath, dpi=150, bbox_inches="tight")
    plt.close(fig)
    print("Saved:", outpath)

Saved: plots/plot_1_2_marrln.png
